In [2]:
pip install tensorflow.io


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 12.7 MB/s eta 0:00:00


In [45]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np

In [58]:
wale = os.path.join('data', '5601601O.wav')
not_wale = os.path.join('data', '81015005.wav')
test = os.path.join('data', 'Eubalaena_glacialis', '5601700F.wav')

In [40]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [54]:
EG = os.path.join('data', 'Eubalaena_glacialis')
PY = os.path.join('data', 'Physalus')

In [55]:
eg_dt = tf.data.Dataset.list_files(EG+'/*.wav')
py_dt = tf.data.Dataset.list_files(PY+'/*.wav')

In [56]:
eu_gl = tf.data.Dataset.zip((eg_dt, tf.data.Dataset.from_tensor_slices(tf.ones(len(eg_dt)))))
py_gl = tf.data.Dataset.zip((py_dt, tf.data.Dataset.from_tensor_slices(tf.ones(len(py_dt)))))

In [68]:
data = eu_gl.concatenate(py_gl)

lengths = []
for file in os.listdir(os.path.join('data', 'Eubalaena_glacialis')):
    tensor_wave = load_wav_16k_mono(os.path.join('data', 'Eubalaena_glacialis', file))
    lengths.append(len(tensor_wave))

In [63]:
tf.math.reduce_mean(lengths)

<tf.Tensor: shape=(), dtype=int32, numpy=35235>

In [64]:
tf.math.reduce_min(lengths)

<tf.Tensor: shape=(), dtype=int32, numpy=9493>

In [65]:
tf.math.reduce_max(lengths)

<tf.Tensor: shape=(), dtype=int32, numpy=73085>

In [66]:
def preprocess(file_path, label):
    wav = load_wav_16k_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

In [67]:
filepath, label = positives.shuffle(buffer_size=10000).as_numpy_iterator().next()

NameError: name 'positives' is not defined